In [ ]:
### lendo base de cobrança GEVENUE e carregando pelo INDEX somente oq é util pra higienizar
import os
import pandas as pd
import glob
from datetime import datetime


# Configuração para exibir todas as colunas no DataFrame
pd.set_option("display.max_columns", None)

# Definindo os nomes das colunas
header_novo_gevenue = [
    "CÓDIGO DA CONTA", "SENHA", "PRODUTO", "NOME", "TIPO DE PESSOA", "CPF/CNPJ",
    "DATA_NASCIMENTO", "FILA_COBRANCA", "ULT_DATA_SMS", "ULT_DATA_CARTA",
    "ULT_DATA_URA", "ULT_DATA_EMAIL", "CARTEIRA (REGIÃO)", "VENCIMENTO",
    "Valor Claro Móvel", "Valor Claro Residencial", "Valor Claro Empresarial",
    "Valor Claro Fixo", "Valor Claro TV", "Valor TvExpurgo", "Valor Oi",
    "DATA_NEGATIVACAO", "DATA_CADASTRO", "STATUS_CONTRATO_NET", "EMAIL_01",
    "EMAIL_02", "EMAIL_03", "TIPO_DE_ENDERECO_1", "ORIGEM_ENDERECO_1",
    "CIDADE_END_INS", "ESTADO_END_INS", "CEP_END_INS", "TIPO_DE_ENDERECO_2",
    "ORIGEM_ENDERECO_2", "CIDADE_END_FAT", "UF_END_FAT", "CEP_END_FAT",
    "DDD_FIXO_1", "TEL_FIXO_1", "DDD_FIXO_2", "TEL_FIXO_2", "DDD_FIXO_3",
    "TEL_FIXO_3", "DDD_FIXO_4", "TEL_FIXO_4", "DDD_CEL_1", "TEL_CEL_1",
    "DDD_CEL_2", "TEL_CEL_2", "DDD_CEL_3", "TEL_CEL_3", "DDD_CEL_4", "TEL_CEL_4"
]

# Definindo o caminho para a pasta onde estão os arquivos .txt
caminho = r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\BASE DE COBRANÇA\Base de Cobrança GEV"
arquivos = glob.glob(os.path.join(caminho, "*.txt"))

# Lista de larguras do layout NET COB
lista_de_larguras = [
    15, 12, 15, 40, 8, 15, 10, 50, 16, 16, 16, 16, 30, 10, 12, 12, 12, 12, 12, 12,
    12, 10, 10, 50, 50, 50, 50, 10, 300, 40, 2, 9, 10, 300, 40, 2, 9, 2, 10, 2,
    10, 2, 10, 2, 10, 2, 10, 2, 10, 2, 10, 2, 10, 2, 10, 2, 10
]

# Definindo os índices das colunas de interesse
colunas_de_interesse_indices = [0,1,2,3,5,13,24,25,26,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52]

# Selecionando as larguras e os nomes das colunas de interesse
larguras_interesse = [lista_de_larguras[i] for i in colunas_de_interesse_indices]
colunas_interesse = [header_novo_gevenue[i] for i in colunas_de_interesse_indices]

# Calculando as larguras cumulativas para pular colunas desnecessárias
sum_larguras = 0
colunas_interesse_larguras = []
for i in range(len(lista_de_larguras)):
    if i in colunas_de_interesse_indices:
        colunas_interesse_larguras.append((sum_larguras, lista_de_larguras[i]))
    sum_larguras += lista_de_larguras[i]

# Função personalizada para ler apenas as colunas de interesse
def ler_colunas_interesse(linha):
    valores = []
    for inicio, largura in colunas_interesse_larguras:
        valores.append(linha[inicio:inicio+largura].strip())
    return valores

# Lista para armazenar os DataFrames de cada arquivo
dfList = []

# Iterar sobre os arquivos na pasta
for arquivo in arquivos:
    print(f"Lendo arquivo: {arquivo}")
    
    # Lendo o arquivo e selecionando as colunas de interesse
    with open(arquivo, 'r', encoding='utf-8') as file:
        dados = [ler_colunas_interesse(linha) for linha in file]

    # Convertendo os dados para um DataFrame do pandas
    df = pd.DataFrame(dados, columns=colunas_interesse)
    dfList.append(df)

# Concatenar todos os DataFrames em um único DataFrame
base_cob = pd.concat(dfList, ignore_index=True)

In [ ]:
## Filtrando apenas quem NÃO TEM TELEFONE 
base_sem_tel = base_cob.query("TEL_FIXO_1 == '' and TEL_FIXO_2 == '' and TEL_FIXO_3 == '' and TEL_FIXO_4 == '' and TEL_CEL_1 == '' and TEL_CEL_2 == '' and TEL_CEL_3 == '' and TEL_CEL_4 == '' ")
base_sem_tel

In [ ]:
# Converter a coluna de vencimento para datetime
base_sem_tel['VENCIMENTO'] = pd.to_datetime(base_sem_tel['VENCIMENTO'], format='%d/%m/%Y')

# Obter a data atual
data_atual = datetime.now()

# Calcular os dias em atraso
base_sem_tel['ATRASO'] = (data_atual - base_sem_tel['VENCIMENTO']).dt.days

# Substituir valores negativos por zero (caso não estejam em atraso)
base_sem_tel['ATRASO'] = base_sem_tel['ATRASO'].apply(lambda x: x if x > 0 else 0)


## Removendo duplicado por cpf e organizando o layout
base_sem_tel = base_sem_tel[['CÓDIGO DA CONTA', 'SENHA', 'PRODUTO', 'NOME', 'CPF/CNPJ', 'VENCIMENTO','ATRASO','EMAIL_01', 'EMAIL_02', 'EMAIL_03']].drop_duplicates(['CPF/CNPJ'])

### Filtrando a faixa que podem cobrar
base_sem_tel = base_sem_tel.query("ATRASO < 1785")

In [ ]:
base_sem_tel.to_csv(r"R:\TI\TELEFONIA\Mailings Prontos\2024\Agosto\05\Jorgean/base_sem_tel_para_higienizar.csv",index=False,sep=';')